In [1]:
import copy
import re
from collections import defaultdict
import pickle

import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import numpy as np

In [2]:
topics = pd.read_csv("../topic_words.csv.zip")
topics.head(10)

,meta,topic,weight,type,word
0,env,Экология в целом,5,0,воздействие
1,env,Экология в целом,5,0,окружающая среда
2,env,Экология в целом,5,0,природа
3,env,Экология в целом,5,0,природоохранный
4,env,Экология в целом,5,0,экологический
5,env,Экология в целом,5,0,экологичный
6,env,Экология в целом,1,0,влияние
7,env,Экология в целом,1,0,документ
8,env,Экология в целом,1,0,документация
9,env,Экология в целом,1,0,защита


In [3]:
topics["topic"].value_counts()
pass

In [4]:
re_dict = {}
re_advanced_dict = {}
sheets_dict = {}
sheets_advanced_dict = {}
base_dict = {}
advanced_dict = {}
importance_dict = {}

for idx, row in topics.iterrows():
    if not sheets_dict.get(row["topic"], False):
        sheets_dict[row["topic"]] = {}
        sheets_advanced_dict[row["topic"]] = {}
        base_dict[row["topic"]] = {}
        advanced_dict[row["topic"]] = {}
    if row["word"].find(",") != -1:
        continue
    sheets_dict[row["topic"]][row["word"]] = row["weight"]  # row["type"]
    if row["type"] == 1:
        sheets_advanced_dict[row["topic"]][row["word"]] = row["weight"]
        advanced_dict[row["topic"]][row["word"]] = 0
    base_dict[row["topic"]][row["word"]] = 0
    importance_dict[row["topic"]] = 0

In [5]:
for key, val in sheets_dict.items():
    re_dict[key] = re.compile("((?<!\w)" + "(?!\w)|(?<!\w)".join(val.keys()) + "(?!\w))")

In [6]:
for key, val in sheets_advanced_dict.items():
    re_advanced_dict[key] = re.compile("((?<!\w)" + "(?!\w)|(?<!\w)".join(val.keys()) + "(?!\w))")

# Feature matrix

In [7]:
regex_expert = re.compile("(" + "(?!\w)|".join(topics["word"].unique()) + "(?!\w))")

In [8]:
label2idx = {topic: i for i, topic in enumerate(sorted(topics["topic"].unique()))}
idx2label = {i: topic for i, topic in enumerate(sorted(topics["topic"].unique()))}
idx2word = {i: word for i, word in enumerate(topics["word"].tolist())}
word2idx = defaultdict(list)

for i, word in enumerate(topics["word"].tolist()):
    word2idx[word].append(i)

In [9]:
topics_vectors = np.zeros((topics["topic"].nunique(), topics.shape[0]), dtype=int)

for topic, topic_words in topics.groupby("topic")["word"].agg(list).items():
    for word in topic_words:
        for idx in word2idx[word]:
            topics_vectors[label2idx[topic], idx] = 1

In [10]:
df = pd.read_csv("etalon/etalon_paragraphs.csv", on_bad_lines='skip')
df

,Unnamed: 0,original_text,cleaned_text,true_expert
0,0,в соответствии с внутренними документами компа...,соответствие внутренний документ компания стру...,Корпоративное управление
1,1,в 2022 году компания продолжила совершенствова...,2022 год компания продолжить совершенствовать ...,Корпоративное управление
2,2,на прошедших в 2022 году собраниях успешно при...,прошедшее 2022 год собрание успешно применятьс...,Корпоративное управление
3,3,при организации системы корпоративного управле...,организация система корпоративный управление к...,Корпоративное управление
4,4,система корпоративного управления выстроена с ...,корпоративный управление выстроить учёт рекоме...,Корпоративное управление
...,...,...,...,...
1420,1420,рост регулируемых тарифов сетевых организаций ...,рост регулировать тариф сетевой организация ка...,Потребители. Ценовая политика
1421,1421,втб стремится обеспечить доступное жилищное кр...,стремиться обеспечить доступный жилищный креди...,Потребители. Ценовая политика
1422,1422,приобрести автомобиль в рамках госпрограммы мо...,приобрести автомобиль рамка госпрограмма мочь ...,Потребители. Ценовая политика
1423,1423,втб выдает кредиты на льготных условиях в рамк...,выдавать кредит льготный условие рамка государ...,Потребители. Ценовая политика


In [11]:
vector = []
labels = []
found_words = []
true_expert_found_words = []
texts = []

for i, (text, expert_topic, original_text) in tqdm(
    enumerate(zip(df["cleaned_text"], df["true_expert"], df["original_text"])), total=df["cleaned_text"].shape[0]
):
    tmp = np.zeros(topics_vectors.shape[1])
    if pd.isna(text):
        continue

    if not re_dict.get(expert_topic, False):
        continue

    if len(re_dict[expert_topic].findall(text)) < 5:
        continue

    texts.append({"text": text, "topic": expert_topic, "original_text": original_text})

    text_words = {}
    for key, regex in re_dict.items():
        words = regex.findall(text)
        text_words[key] = words
        if key == expert_topic:
            true_expert_found_words.append((key, words))
        for word in words:
            tmp[word2idx[word]] = 1
    vector.append(tmp)
    labels.append(expert_topic)
    found_words.append(text_words)

100%|██████████████████████████████████████| 1425/1425 [00:11<00:00, 119.30it/s]


In [12]:
pd.DataFrame(texts).to_csv("etalon/et_used_texts_expert.csv")

In [14]:
vector = np.vstack(vector)

In [15]:
w = pd.DataFrame(found_words)
w.head(5)

,Экология в целом,Климат,Энергия,Воздух,Вода,Отходы и циклическая экономика,Биоразнообразие,Рекультивация земель,Экологичность продукта,!Персонал в целом,...,Отчетность и прозрачность,Отношения с инвесторами,!Инновации,!Кибербезопасность,Права человека,Лидерство,Риски,Этика и антикоррупция,Корпоративное управление,Устойчивое развитие
0,"[документ, управление, управление, управление,...",[],"[общий, высокий, решение, общий, общий, работа...","[управление, управление, управление]",[структура],[защита],[],[контроль],[эффективный],"[работа, работа]",...,"[внутренний, директор, директор, директор, дир...","[собрание, акционер, деятельность, деятельност...",[],[],"[оперативный, обеспечивать, обеспечивать, право]","[собрание, совет, директор, руководство, компе...","[основный, предварительный, важный, защита]",[комитет],"[собрание акционер, совет директор, компетенци...","[соответствие, деятельность, деятельность, дея..."
1,[],"[(учёт, )]","[общий, общий]",[смочь],[учёт],"[электронный, электронный, электронный]",[],[],[],[],...,[],"[собрание, акционер, акционер, акционер, акцио...",[],[],[право],"[собрание, годовой, собрание, собрание]",[],[],"[практика, акционер, акционер, акционер, акцио...",[дополнительный]
2,"[система, управление, управление, управление, ...","[(реализация, ), (рыночный, )]","[система, высокий]","[управление, управление, управление, управлени...",[поставщик],"[повышение, стоимость]",[международный],"[российский, закрепить]",[эффективный],[отношение],...,[принцип],"[деятельность, прозрачный, акционер, равный, о...",[],[],"[кодекс, кодекс, кодекс, закрепить, механизм, ...","[кодекс, кодекс, кодекс, принцип, сотрудник]",[сторона],"[кодекс, кодекс, кодекс]","[хороший, российский практика, акционер, акцио...",[деятельность]
3,"[управление, управление, управление, управлени...","[(учёт, )]",[общий],"[управление, управление, управление, управлени...","[учёт, структура]","[устойчивый, развитие]",[],[российский],[],[],...,"[директор, директор, директор]","[листинг, московский, биржа, собрание, акционер]",[],[],"[кодекс, развитие]","[кодекс, собрание, совет, директор, генеральны...",[],[кодекс],"[хороший, российский практика, собрание акцион...",[управление устойчивый развитие]
4,"[система, управление, система, управление, упр...","[(стратегия, ), (стратегия, ), (реализация, ),...","[система, система, решение, работа, система, в...","[управление, управление, управление, управлени...",[],"[повышать, защита, развитие, повышение]",[существенный],"[российский, законодательство, контроль, контр...","[эффективный, эффективный]",[работа],...,"[уверенность, принцип, контроль, контроль, рис...","[акционер, общество]",[],[риск],"[обеспечивать, право, развитие, риск, предотвр...","[принятие, принцип, принятие, этический принци...","[уверенность, сторона, защита, управление риск...",[конфликт интерес],"[хороший, практика, акционер, ответственность,...","[соответствие, стратегия, стратегия, риск, стр..."


In [16]:
p = pd.DataFrame(true_expert_found_words).rename(columns={0: "topic", 1: "words"})
p.head()

,topic,words
0,Корпоративное управление,"[собрание акционер, совет директор, компетенци..."
1,Корпоративное управление,"[практика, акционер, акционер, акционер, акцио..."
2,Корпоративное управление,"[хороший, российский практика, акционер, акцио..."
3,Корпоративное управление,"[хороший, российский практика, собрание акцион..."
4,Корпоративное управление,"[хороший, практика, акционер, ответственность,..."


In [17]:
print(f"{'Topic':<35} {'Average Word Length':<20} Count")
for topic, val in p.groupby("topic")["words"].apply(list).items():
    print(f"{topic:<35} {sum([len(e) for e in val]) / len(val):05.2f} {len(val)}")

Topic                               Average Word Length  Count
Биоразнообразие                     08.58 26
Вода                                13.52 27
Воздух                              07.85 20
Климат                              10.52 25
Корпоративное управление            07.00 9
Лидерство                           05.00 1
Отношения с инвесторами             08.18 11
Отчетность и прозрачность           07.30 10
Потребители. Доступность            06.85 13
Потребители. Ценовая политика       07.50 14
Права человека                      11.29 24
Риски                               10.00 22
Сотрудники. Наем и увольнение       06.00 2
Экологичность продукта              06.71 7
Экология в целом                    08.72 18
Энергия                             10.88 26
Этика и антикоррупция               06.65 17


In [18]:
def init_matrix(topics: pd.DataFrame) -> np.ndarray:
    matrix = np.zeros(topics_vectors.shape)
    # topic	weight	type	word
    for idx, row in topics.iterrows():
        matrix[label2idx[row["topic"]], word2idx[row["word"]]] = 1
    return matrix


def fit(matrix, vectors, labels):
    for vector, label in zip(vectors, labels):
        for i in range(matrix.shape[0]):
            if not label2idx.get(label, False):
                continue
            #print(label)
            if i == label2idx[label]:
                matrix[i] += vector
            else:
                matrix[i] -= vector
    return matrix


def test(matrix, vectors, labels, debug=False):
    total = 0
    pred_labels = []
    for vector, label in zip(vectors, labels):
        res = np.argmax(vector @ matrix.T)
        # print(np.argmax(vector @ matrix.T))
        total += label == idx2label[res]
        pred_labels.append(idx2label[res])
        if debug and label != idx2label[res]:
            print("expected", label, "result", idx2label[res], vector @ matrix.T)
    return pred_labels, total / len(labels)

In [19]:
matrix = init_matrix(topics)
for i in tqdm(range(2)):
    matrix = fit(matrix, vector, labels)
    pred_labels, acc = test(matrix, vector, labels, False)
    print(acc)

pred_labels, acc = test(matrix, vector, labels, False)
acc

100%|█████████████████████████████████████████████| 2/2 [00:00<00:00, 26.18it/s]

0.7904411764705882
0.7867647058823529


0.7867647058823529

In [20]:
matrix_df = pd.DataFrame(matrix, columns=topics["word"].tolist(), index=topics["topic"].unique().tolist())
matrix_df

,воздействие,окружающая среда,природа,природоохранный,экологический,экологичный,влияние,документ,документация,защита,...,соответствовать,управлять,фокусный,формировать,iso 14001,science based targets initiative sbti,верификация,целевый показатель,система экологического менеджмент,сэм
Экология в целом,-76.0,0.0,-16.0,-20.0,-66.0,-8.0,-46.0,-52.0,-12.0,-32.0,...,0.0,-2.0,0.0,-10.0,-4.0,0.0,-6.0,0.0,0.0,-8.0
Климат,-76.0,0.0,-16.0,-20.0,-66.0,-8.0,-46.0,-52.0,-12.0,-32.0,...,0.0,-2.0,0.0,-10.0,-4.0,0.0,-6.0,0.0,0.0,-8.0
Энергия,-76.0,0.0,-16.0,-20.0,-66.0,-8.0,-46.0,-52.0,-12.0,-32.0,...,0.0,-2.0,0.0,-10.0,-4.0,0.0,-6.0,0.0,0.0,-8.0
Воздух,-35.0,0.0,13.0,-15.0,-53.0,-8.0,-30.0,-52.0,-12.0,-28.0,...,0.0,-2.0,0.0,-10.0,-4.0,0.0,-6.0,0.0,0.0,-8.0
Вода,-51.0,0.0,-16.0,-16.0,-62.0,-4.0,-34.0,-48.0,0.0,-32.0,...,0.0,-2.0,0.0,-10.0,-4.0,0.0,-6.0,0.0,0.0,-8.0
Отходы и циклическая экономика,-59.0,0.0,-16.0,-12.0,-49.0,-8.0,-38.0,-52.0,-12.0,-32.0,...,0.0,-2.0,0.0,-10.0,-4.0,0.0,-6.0,0.0,0.0,-8.0
Биоразнообразие,-76.0,0.0,-16.0,-20.0,-66.0,-8.0,-46.0,-52.0,-12.0,-32.0,...,0.0,-2.0,0.0,-10.0,-4.0,0.0,-6.0,0.0,0.0,-8.0
Рекультивация земель,-76.0,0.0,-16.0,-20.0,-66.0,-8.0,-46.0,-52.0,-12.0,-32.0,...,0.0,-2.0,0.0,-10.0,-4.0,0.0,-6.0,0.0,0.0,-8.0
Экологичность продукта,-71.0,0.0,-16.0,-20.0,-54.0,-4.0,-46.0,-48.0,-12.0,-32.0,...,0.0,-2.0,0.0,-10.0,-4.0,0.0,-2.0,0.0,0.0,-8.0
!Персонал в целом,-76.0,0.0,-16.0,-20.0,-65.0,-8.0,-46.0,-52.0,-12.0,-32.0,...,0.0,-2.0,0.0,-10.0,-4.0,0.0,-6.0,0.0,0.0,-8.0


In [22]:
np.save("etalon/et_expert.npy", matrix)
matrix_df.to_csv("etalon/et_expert_matrix.csv")
matrix_df.T.to_csv("etalon/et_expert_matrix.T.csv")

In [23]:
results = []
for col in tqdm(matrix_df.columns):
    max_val = matrix_df[col].max()[0] if type(matrix_df[col].max()) is not float else matrix_df[col].max()
    min_val = matrix_df[col].min()[0] if type(matrix_df[col].min()) is not float else matrix_df[col].min()
    if max_val > 0:
        results.append((col, max_val))
    if min_val < -2000:
        results.append((col, min_val))

results_df = pd.DataFrame(results, columns=["word", "val"])
results_df.to_csv("etalon/et_matrix_words.csv")

100%|██████████████████████████████████████| 3650/3650 [00:06<00:00, 570.53it/s]


In [24]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

print(
    "accuracy",
    accuracy_score(
        labels,
        pred_labels,
    ),
)
print(
    "f1",
    f1_score(
        labels,
        pred_labels,
        average="macro",
    ),
)
print(
    "precision",
    precision_score(
        labels,
        pred_labels,
        average="macro",
    ),
)
print(
    "recall",
    recall_score(
        labels,
        pred_labels,
        average="macro",
    ),
)

accuracy 0.7867647058823529
f1 0.6545817743978198
precision 0.7123900837985843
recall 0.6502576745310309


/Users/darlinkennedy/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Topic assignment

In [26]:
from typing import Optional
from typing import Union
import typer

In [25]:
df = pd.read_csv("paragraphs/paragraphs_2022.csv")  # , usecols=["cleaned_text"], nrows=ROWS
df = df.dropna().reset_index(drop=True)
df.shape

(14154, 8)

In [27]:
text = df.loc[100, "cleaned_text"]
# text = "Доля сотрудников, охваченных коллективными договорами".lower()


def test_algos(text: str, verbouse: bool = False) -> dict[str, Union[str, float, list[str],None]]:
    arr = []
    result = {"text": text}
    vector = np.zeros(matrix.shape[1])

    for key, regex in re_dict.items():
        words = regex.findall(text)
        paragraph_dict = copy.deepcopy(base_dict)
        #print(paragraph_dict)
        importance = 0
        paragraph_len = len(text.split())
        base_words = 0
        for word in words:
            if type(word) is tuple:
                print(word)
                word = word[0]
            paragraph_dict[key][word] = sheets_dict[key][word]
            importance += sheets_dict[key][word] > 1
            vector[word2idx[word]] = 1
            base_words += 1
        # if base_words / paragraph_len < 0.1 or importance < 1:
        #     arr.append((key, 0, 0, []))
        #     continue

        dist = cosine_similarity(X=[list(sheets_dict[key].values())], Y=[list(paragraph_dict[key].values())])[0][0]
        arr.append((copy.deepcopy(key), copy.deepcopy(dist), copy.deepcopy(importance), copy.deepcopy(words)))
        result = {**result, **{f"{key}_dist": dist, f"{key}_importance": importance, f"{key}_words": words}}
    if verbouse:
        print("\nexpert")
    res = vector @ matrix.T
    for idx in np.argsort(res)[::-1]:
        if verbouse:
            print(idx2label[idx], res[idx])
        result[f"{idx2label[idx]}_expert"] = res[idx]

    if verbouse:
        print("\ncos")
        for key, dist, importance, words in sorted(arr, key=lambda x: x[1], reverse=True):
            print(f"{key} расстояние {dist:.4f} важные слова {importance} {copy.deepcopy(set(words))}")
    return result

In [28]:
text = df.loc[100, "cleaned_text"]
print(text)
test_algos(text, True)
a = 0

наш нераспределённый стоимость 2022 год составить 18,2 млрд руб. увеличиться 152,2 сравнение предыдущий год данный изменение обусловить положительный динамика показатель ebitda число завершение продажа силовой остров снижение ряд расход совместно решение выплачивать дивиденд 2022 год привести рост нераспределённый прибыль
('показатель', '')
('снижение', '')
('рост', '')

expert
Климат -723.0
Вода -727.0
Воздух -760.0
Энергия -777.0
Экология в целом -952.0
Риски -984.0
Потребители. Ценовая политика -1000.0
Биоразнообразие -1072.0
Отношения с инвесторами -1077.0
Корпоративное управление -1080.0
Отчетность и прозрачность -1100.0
Права человека -1100.0
Экологичность продукта -1112.0
Потребители. Сервис и коммуника -1119.0
Сотрудники. Наем и увольнение -1120.0
Потребители. Доступность -1121.0
Отходы и циклическая экономика -1128.0
Потребители. Удовлетворенность -1130.0
Потребители. Качество и безопас -1131.0
Рекультивация земель -1132.0
Коренные народы и местные сообщ -1133.0
Социальные инв

In [29]:
def get_scores(text: str) -> dict[str, Union[str, float, list[str],None]]:
    paragraph_dict = copy.deepcopy(base_dict)
    paragraph_importance = copy.deepcopy(importance_dict)
    found_words = False
    base_words = {}
    advanced_words = {}
    vector_exp = np.zeros(matrix.shape[1])
    if type(text) is float:
        print(text, type(text))
        return {}
    paragraph_len = len(text.split())

    total_found = 0
    for key, regex in re_dict.items():
        words = regex.findall(text)
        # advanced_words_found = re_advanced_dict[key].findall(text)
        for word in words:
            found_words = True
            if type(word) is tuple:
                word = word[0]
            weight = sheets_dict[key][word]
            paragraph_dict[key][word] = weight
            paragraph_importance[key] += weight > 1

            vector_exp[word2idx[word]] = 1

        base_words[key] = len(words)
        total_found += len(words)
        # advanced_words[key] = len(advanced_words_found)

    debug_dict = {}
    if not found_words:
        res = {key: 0 for key in sheets_dict}
        debug_dict["total_found_words"] = total_found
        return {**res, **{"max_score_cos": None, "max_topic_cos": None, "count_words": None}, **debug_dict}

    topic_found = False
    res = {}
    for key, vector in sheets_dict.items():
        debug_dict[f"{key}_percent_of_words"] = base_words[key] / paragraph_len
        debug_dict[f"{key}_importance"] = paragraph_importance[key]
        debug_dict[f"{key}_base_words_count"] = base_words[key]
        if base_words[key] / paragraph_len < 0.1 or paragraph_importance[key] < 1:
            res[key] = 0
            continue
        sim = cosine_similarity(X=[list(vector.values())], Y=[list(paragraph_dict[key].values())])[0][0]
        res[key] = sim
        topic_found = True

    if not topic_found:
        res = {key: 0 for key in sheets_dict}
        return {**res, **{"max_score_cos": -100, "max_topic_cos": "безтемы", "count_words": None}, **debug_dict}

    paragraph_matrix = vector_exp @ matrix.T
    best_indicies = paragraph_matrix.argsort()[-2:][::-1]
    best_topics = [idx2label[i] for i in best_indicies]
    best_scores = paragraph_matrix[best_indicies]

    res_sorted = sorted(res.items(), key=lambda x: x[1], reverse=True)
    max_value = res_sorted[0]
    second_values = res_sorted[1]
    if second_values[1] == 0:
        second_values = (None, None)

    return {
        **res,
        **{
            "max_topic_cos": max_value[0],
            "max_score_cos": max_value[1],
            "second_max_topic_cos": second_values[0],
            "second_max_score_cos": second_values[1],
            "max_topic_exp": best_topics[0],
            "max_score_exp": best_scores[0],
            "second_max_topic_exp": best_topics[1],
            "second_max_score_exp": best_scores[1],
            "total_found_words": total_found,
        },
        **debug_dict,
    }

In [30]:
topics = []

i = 0
for idx, row in tqdm(df.iterrows(), total=df.shape[0]):
    text = row["cleaned_text"]
    res = get_scores(text)
    res["idx"] = idx
    if res.get("max_score_cos") is None:
        continue
    res["cleaned_text"] = text
    topics.append(res)

100%|█████████████████████████████████████| 14154/14154 [08:48<00:00, 26.76it/s]


In [31]:
with open("topics.pkl", "wb") as fp:  # Pickling
    pickle.dump(topics, fp)

In [32]:
with open("topics.pkl", "rb") as f:
    topics = pickle.load(f)

In [33]:
topics = pd.DataFrame(topics).set_index(keys="idx")

In [34]:
topics.to_csv("paragraphs/all_topics_2022.csv")
topics.to_csv("paragraphs/all_topics_2022.csv.zip")

In [35]:
df = df.join(topics, rsuffix="topics")
df.head(10)

,Unnamed: 0,rsspp_index,index,company,paragraph,original_text,cleaned_text,year,Экология в целом,Климат,...,Устойчивое развитие_importance,Устойчивое развитие_base_words_count,cleaned_texttopics,second_max_topic_cos,second_max_score_cos,max_topic_exp,max_score_exp,second_max_topic_exp,second_max_score_exp,total_found_words
0,0,1428,1428,ЭЛ5-Энерго 2022,42,алибек айбекович тналин генеральный директор э...,алибек айбекович тналин генеральный директор у...,2022,0.0,0.0,...,0.0,0.0,алибек айбекович тналин генеральный директор у...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1428,1428,ЭЛ5-Энерго 2022,43,"в завершение хочу еще раз отметить, что данные...",завершение хотеть отметить дать достижение явл...,2022,0.0,0.0,...,2.0,2.0,завершение хотеть отметить дать достижение явл...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1428,1428,ЭЛ5-Энерго 2022,47,представляем вашему вниманию наш второй отчет ...,представлять ваш внимание наш второй отчёт уст...,2022,0.0,0.0,...,1.0,3.0,представлять ваш внимание наш второй отчёт уст...,None,NaN,Отчетность и прозрачность,-862.0,Риски,-902.0,26.0
3,3,1428,1428,ЭЛ5-Энерго 2022,48,наша гордость в 2022 году достижение показател...,наш гордость 2022 год достижение показатель но...,2022,0.0,0.0,...,0.0,0.0,наш гордость 2022 год достижение показатель но...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1428,1428,ЭЛ5-Энерго 2022,50,мы прилагаем большие усилия для развития и мот...,прилагать больший усилие развитие мотивация со...,2022,0.0,0.0,...,0.0,1.0,прилагать больший усилие развитие мотивация со...,None,NaN,Права человека,-1674.0,Экология в целом,-1866.0,57.0
5,5,1428,1428,ЭЛ5-Энерго 2022,51,мы осознаем свою ответственность и делаем все ...,осознать ответственность делать возможный выпо...,2022,0.0,0.0,...,0.0,0.0,осознать ответственность делать возможный выпо...,Лидерство,0.233224,Экология в целом,-626.0,Права человека,-652.0,34.0
6,6,1428,1428,ЭЛ5-Энерго 2022,52,2022 года мы можем с гордостью отметить достиж...,2022 год мочь гордость отметить достижение важ...,2022,0.0,0.0,...,0.0,1.0,2022 год мочь гордость отметить достижение важ...,None,NaN,Экология в целом,-1212.0,Климат,-1256.0,37.0
7,7,1428,1428,ЭЛ5-Энерго 2022,53,"20232025 годов, их предыдущие версии неоднокра...",20232025 год предыдущий версия неоднократно пр...,2022,0.0,0.0,...,0.0,0.0,20232025 год предыдущий версия неоднократно пр...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,1428,1428,ЭЛ5-Энерго 2022,55,мы не только нарастили объем социальных инвест...,нарастить объём социальный инвестиция сравнени...,2022,0.0,0.0,...,1.0,2.0,нарастить объём социальный инвестиция сравнени...,Обучение и развитие,0.245327,Отчетность и прозрачность,-858.0,Вода,-876.0,42.0
9,9,1428,1428,ЭЛ5-Энерго 2022,62,принципы подготовки отчетности отчет об устойч...,принцип подготовка отчётность отчёт устойчивый...,2022,0.0,0.0,...,2.0,6.0,принцип подготовка отчётность отчёт устойчивый...,Обучение и развитие,0.254588,Экология в целом,-2908.0,Риски,-3183.0,92.0


In [36]:
df.to_csv("paragraphs/paragrapghs_with_topisc_2022.csv")
df.to_csv("paragraphs/paragrapghs_with_topisc_2022.csv.zip")